<div class="alert alert-success">
<h1> Beer Data Science Assignment </h1>
</div>


In [17]:
from IPython.display import HTML
import pandas as pd
import os
import numpy as np
from datetime import datetime
import wordcloud
import warnings
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
warnings.filterwarnings('ignore')

BASE_PATH = "../"
INPUT_DATA_PATH = os.path.join(BASE_PATH, "data/raw/BeerDataScienceProject.csv")

In [2]:
df_beer_raw = pd.read_csv(INPUT_DATA_PATH, encoding="ISO-8859-1")

In [3]:
df_beer_raw.shape

(528870, 13)

In [4]:
df_beer_raw.dtypes

beer_ABV              float64
beer_beerId             int64
beer_brewerId           int64
beer_name              object
beer_style             object
review_appearance     float64
review_palette        float64
review_overall        float64
review_taste          float64
review_profileName     object
review_aroma          float64
review_text            object
review_time             int64
dtype: object

### Data Cleaning
```In the below cell we can see that there are some columns where data is missing. 
   We need to remove those data points from our data set.
```

In [5]:
df_beer_raw.isna().sum()

beer_ABV              20280
beer_beerId               0
beer_brewerId             0
beer_name                 0
beer_style                0
review_appearance         0
review_palette            0
review_overall            0
review_taste              0
review_profileName      115
review_aroma              0
review_text             119
review_time               0
dtype: int64

In [6]:
df_beer = df_beer_raw.dropna()

In [7]:
df_beer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508358 entries, 0 to 528796
Data columns (total 13 columns):
beer_ABV              508358 non-null float64
beer_beerId           508358 non-null int64
beer_brewerId         508358 non-null int64
beer_name             508358 non-null object
beer_style            508358 non-null object
review_appearance     508358 non-null float64
review_palette        508358 non-null float64
review_overall        508358 non-null float64
review_taste          508358 non-null float64
review_profileName    508358 non-null object
review_aroma          508358 non-null float64
review_text           508358 non-null object
review_time           508358 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 54.3+ MB


## Data Changes in Raw and Clean dataset

In [8]:
print("Raw data size: ", df_beer_raw.shape[0])
print("Clean data size: ", df_beer.shape[0])
print("Data points difference: ", df_beer_raw.shape[0]-df_beer.shape[0])

Raw data size:  528870
Clean data size:  508358
Data points difference:  20512


### Describe the categorical features

In [9]:
df_beer.describe(include=[np.object])

,beer_name,beer_style,review_profileName,review_text
count,508358,508358,508358,508358
unique,14028,104,22563,508007
top,Sierra Nevada Celebration Ale,American IPA,northyorksammy,#NAME?
freq,2998,42092,1732,90


### Describe the numerical features

In [10]:
df_beer.describe(include=[np.number])

,beer_ABV,beer_beerId,beer_brewerId,review_appearance,review_palette,review_overall,review_taste,review_aroma,review_time
count,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,5.083580e+05
mean,7.017400,21824.120840,2534.273207,3.872676,3.768993,3.840805,3.775327,3.827646,1.226175e+09
std,2.204528,22124.969107,5237.843825,0.601764,0.682355,0.706408,0.665586,0.715121,7.530843e+07
min,0.010000,5.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,8.843904e+08
25%,5.300000,1673.000000,132.000000,3.500000,3.500000,3.500000,3.500000,3.500000,1.177202e+09
50%,6.500000,13850.000000,392.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.241503e+09
75%,8.500000,40418.000000,1315.000000,4.000000,4.000000,4.500000,4.000000,4.500000,1.289073e+09
max,57.700000,77310.000000,27980.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.326277e+09


In [11]:
NUMERICAL_ATTRIBUTE = ['beer_ABV', 'review_appearance', 'review_palette', 
                       'review_overall', 'review_taste', 'review_aroma', 'review_time']
CATEGORICAL_ATTRIBUTE = ['beer_name', 'beer_style', 'review_profileName', 'review_text']

In [12]:
df_beer.sample(4)

,beer_ABV,beer_beerId,beer_brewerId,beer_name,beer_style,review_appearance,review_palette,review_overall,review_taste,review_profileName,review_aroma,review_text,review_time
289283,9.2,47360,35,Samuel Adams Imperial Stout,Russian Imperial Stout,4.5,4.0,3.0,4.0,Naugled,4.0,12oz bottle served in tulip glass A: thick bro...,1240498698
1127,3.5,436,163,Amstel Light,Light Lager,2.5,2.0,2.0,2.5,DoubleJ,2.0,"What's with all the fuss over this beer? Well,...",1184607136
80413,5.6,279,140,Sierra Nevada Porter,American Porter,4.0,3.0,4.0,4.0,RJLarse,4.5,I picked up a couple of bottles of Sierra Neva...,1163376203
356653,5.9,6368,1177,Masala Mama India Pale Ale,American IPA,5.0,4.5,4.5,4.0,kbeals,4.0,Growler sent to me by younger35. A- Dark amber...,1273412648


<div class="alert alert-info">
Convert the review timestamp to datetime format
</div>

In [13]:
df_beer['d_review_time'] = df_beer.review_time.apply(lambda t: datetime.fromtimestamp(t))

In [14]:
df_beer['d_review_time_year'] = df_beer['d_review_time'].dt.year
df_beer['d_review_time_month'] = df_beer['d_review_time'].dt.month

<div class="alert alert-success">
Q7) How do find similar beer drinkers by using written reviews only?   
</div>


In [18]:
import torch
import transformers as ppb

model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


In [19]:
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

I0531 18:28:04.546885 140234029037376 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/chetan/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0531 18:28:06.023221 140234029037376 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at /home/chetan/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
I0531 18:28:06.026210 140234029037376 configuration_utils.py:321] Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_

I0531 18:30:58.036954 140234029037376 file_utils.py:440] storing https://cdn.huggingface.co/distilbert-base-uncased-pytorch_model.bin in cache at /home/chetan/.cache/torch/transformers/ae9df7a8d658c4f3e1917a471a8a21cf678fa1d4cb91e7702dfe0598dbdcf354.c2015533705b9dff680ae707e205a35e2860e8d148b45d35085419d74fe57ac5
I0531 18:30:58.039834 140234029037376 file_utils.py:443] creating metadata file for /home/chetan/.cache/torch/transformers/ae9df7a8d658c4f3e1917a471a8a21cf678fa1d4cb91e7702dfe0598dbdcf354.c2015533705b9dff680ae707e205a35e2860e8d148b45d35085419d74fe57ac5
I0531 18:30:58.043862 140234029037376 filelock.py:318] Lock 140230454146720 released on /home/chetan/.cache/torch/transformers/ae9df7a8d658c4f3e1917a471a8a21cf678fa1d4cb91e7702dfe0598dbdcf354.c2015533705b9dff680ae707e205a35e2860e8d148b45d35085419d74fe57ac5.lock
I0531 18:30:58.045972 140234029037376 modeling_utils.py:650] loading weights file https://cdn.huggingface.co/distilbert-base-uncased-pytorch_model.bin from cache at /home

In [ ]:
tokenized = df_beer['review_text'].progress_apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

W0531 18:31:24.101449 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (579 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:24.136695 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:24.997313 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (588 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:26.435384 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (586 > 512). Running this sequence through the model will result in indexing errors
W0531 18

W0531 18:31:31.703445 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:31.730535 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (566 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:32.271724 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (652 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:32.327774 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors
W0531 18

W0531 18:31:36.870691 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (813 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:36.928255 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (586 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:36.969678 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (632 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:38.065981 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (552 > 512). Running this sequence through the model will result in indexing errors
W0531 18

W0531 18:31:45.759685 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (581 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:46.681937 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (1012 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:48.030011 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (621 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:48.661880 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (1129 > 512). Running this sequence through the model will result in indexing errors
W0531 

W0531 18:31:59.862231 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (730 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:59.884137 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (561 > 512). Running this sequence through the model will result in indexing errors
W0531 18:31:59.929752 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (631 > 512). Running this sequence through the model will result in indexing errors
W0531 18:32:00.291922 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors
W0531 18

...

W0531 18:48:56.523749 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (621 > 512). Running this sequence through the model will result in indexing errors
W0531 18:48:56.722252 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (555 > 512). Running this sequence through the model will result in indexing errors
W0531 18:48:56.793153 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (536 > 512). Running this sequence through the model will result in indexing errors
W0531 18:48:57.085441 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (709 > 512). Running this sequence through the model will result in indexing errors
W0531 18

W0531 18:49:10.740348 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (659 > 512). Running this sequence through the model will result in indexing errors
W0531 18:49:10.781272 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (909 > 512). Running this sequence through the model will result in indexing errors
W0531 18:49:10.830339 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (524 > 512). Running this sequence through the model will result in indexing errors
W0531 18:49:10.896939 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors
W0531 18

W0531 18:49:15.129358 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (650 > 512). Running this sequence through the model will result in indexing errors
W0531 18:49:15.381384 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (614 > 512). Running this sequence through the model will result in indexing errors
W0531 18:49:15.513591 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (564 > 512). Running this sequence through the model will result in indexing errors
W0531 18:49:15.604538 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (511 > 512). Running this sequence through the model will result in indexing errors
W0531 18

W0531 18:49:19.565897 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors
W0531 18:49:19.772792 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors
W0531 18:49:19.982968 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (739 > 512). Running this sequence through the model will result in indexing errors
W0531 18:49:20.138754 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (538 > 512). Running this sequence through the model will result in indexing errors
W0531 18

W0531 18:49:26.304068 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (760 > 512). Running this sequence through the model will result in indexing errors
W0531 18:49:26.362171 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (518 > 512). Running this sequence through the model will result in indexing errors
W0531 18:49:26.406711 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (662 > 512). Running this sequence through the model will result in indexing errors
W0531 18:49:26.914590 140234029037376 tokenization_utils.py:1934] Token indices sequence length is longer than the specified maximum sequence length for this model (640 > 512). Running this sequence through the model will result in indexing errors
W0531 18

## Padding
After tokenization, tokenized is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

## Masking
If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

The model() function runs our sentences through BERT. The results of the processing will be returned into last_hidden_states.

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
## Create dataset for faiss indexing

In [ ]:
list_train = [features[i:i+100] for i in range(0,len(features),100)]

In [ ]:
import faiss

dimension = 512
nlist = 5  # number of clusters
quantiser = faiss.IndexFlatL2(dimension)  
index = faiss.IndexIVFFlat(quantiser, dimension, nlist, faiss.METRIC_L2)


In [ ]:
for x in list_train:
    db_vectors = embed(x).numpy()
    print(index.is_trained)   # False
    index.train(db_vectors)  # train on the database vectors
    print(index.ntotal)   # 0
    index.add(db_vectors)   # add the vectors and update the index
    print(index.is_trained)  # True
    print(index.ntotal)